# How happy are Zürcher with Tempo 30?

Dataset from [Daten der Bevölkerungsbefragung der Stadt Zürich 1999 bis 2019](https://data.stadt-zuerich.ch/dataset/prd_stez_bevoelkerungsbefragungen_seit1999_od4730)

We want to compare this with the kilometers (or number of roads) with Tempo 30 in each Kreis :
- [Verkehrszonen](https://www.ogd.stadt-zuerich.ch/geodaten/Verkehrszonen?format=10009)
- [Strassennamenverzeichnis](https://www.ogd.stadt-zuerich.ch/geodaten/download/Strassennamenverzeichnis) (`geoz.sv_str_verz.json`; `geoz.sv_str_lin.json` hat alle strassen mit Koordinaten)

---

You need to unzip `bev473od4731.zip` before running this notebook.

In [132]:
import pandas as pd

In [133]:
df = pd.read_csv('bev473od4731.csv')
df.shape

(28370, 318)

In [134]:
df.head()

,StichtagDatJahr,intnr,methode,fbversion,sgewicht,pgewicht,kreis,s1,f1,f2,...,f58_8,f58_9,f58_10,f58_11,f58_12,f58_13,f59,f60a,f60b,f60c
0,2019,1101,online,Unbekannt,0.279589,17.750000,Kreis 01/02,Frau,Eher gerne,Note 4,...,Ja,Nein,Nein,Nein,Nein,Nein,Sehr zufrieden (6),Sehr zufrieden (6),Sehr zufrieden (6),Sehr zufrieden (6)
1,2019,1102,online,Unbekannt,1.020983,64.818182,Kreis 07,Frau,Eher gerne,Note 5,...,Nein,Nein,Nein,Nein,Nein,Nein,3,4,3,3
2,2019,1103,online,Unbekannt,0.551302,35.000000,Kreis 01/02,Frau,Sehr gerne,Note 6 (sehr gut),...,Ja,Nein,Nein,Nein,Nein,Nein,Sehr zufrieden (6),4,5,5
3,2019,1104,online,Unbekannt,0.989136,62.796296,Kreis 09,Frau,Sehr ungern,Note 6 (sehr gut),...,Nein,Nein,Nein,Nein,Nein,Nein,4,5,5,5
4,2019,1105,online,Unbekannt,0.986286,62.615385,Kreis 01/02,Frau,Sehr gerne,Note 5,...,Nein,Nein,Nein,Nein,Ja,Nein,3,3,3,4


- [Renaming columns](https://stackoverflow.com/a/46096346/5239250)

In [135]:
happy = df[['kreis', 'f1', 'f4l']].rename(columns={'f1': 'happy_zh', 'f4l': 'happy_30'})

happy.head()

,kreis,happy_zh,happy_30
0,Kreis 01/02,Eher gerne,Weiss nicht
1,Kreis 07,Eher gerne,Weiss nicht
2,Kreis 01/02,Sehr gerne,Note 4
3,Kreis 09,Sehr ungern,Note 6 (sehr zufrieden)
4,Kreis 01/02,Sehr gerne,Note 5


https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/

In [136]:
happy.happy_zh.unique()

array(['Eher gerne', 'Sehr gerne', 'Sehr ungern', 'Eher ungern',
       'Keine Angabe'], dtype=object)

In [137]:
happy = happy[happy.happy_zh != 'Keine Angabe']
categories = {'happy_zh': {'Sehr gerne': 4, 'Eher gerne': 3, 'Eher ungern': 2, "Sehr ungern": 1}}
happy.replace(categories, inplace=True)
happy.head()

,kreis,happy_zh,happy_30
0,Kreis 01/02,3,Weiss nicht
1,Kreis 07,3,Weiss nicht
2,Kreis 01/02,4,Note 4
3,Kreis 09,1,Note 6 (sehr zufrieden)
4,Kreis 01/02,4,Note 5


- [How do I remove unwanted parts from strings in a column?](https://stackoverflow.com/a/54302517/5239250)

Removing all but the digits leaves us with a number as string, which we have to convert to numeric.

In [138]:
happy.happy_30.unique()

array(['Weiss nicht', 'Note 4', 'Note 6 (sehr zufrieden)', 'Note 5',
       'Note 2', 'Note 3', 'Note 1 (sehr unzufrieden)', 'Keine Angabe'],
      dtype=object)

In [139]:
happy = happy[happy.happy_30 != "Weiss nicht"]
happy = happy[happy.happy_30 != "Keine Angaben"]
happy.happy_30 = happy.happy_30.str.extract('(\d+)', expand=False)
happy.happy_30 = pd.to_numeric(happy.happy_30)
happy


,kreis,happy_zh,happy_30
2,Kreis 01/02,4,4.0
3,Kreis 09,1,6.0
4,Kreis 01/02,4,5.0
6,Kreis 10,4,4.0
7,Kreis 11,4,2.0
...,...,...,...
28365,Kreis 12,3,5.0
28366,Kreis 12,4,6.0
28367,Kreis 12,4,6.0
28368,Kreis 12,4,4.0


In [140]:
happy.groupby('kreis').mean()

,happy_zh,happy_30
kreis,,
Kreis 01/02,3.757342,4.309773
Kreis 03,3.751210,4.470559
Kreis 04,3.702027,4.521555
Kreis 05,3.753425,4.518968
Kreis 06,3.783482,4.506131
Kreis 07,3.778970,4.325750
Kreis 08,3.803717,4.248266
Kreis 09,3.708949,4.296989
Kreis 10,3.777013,4.337150


happy_zh: 1-4; happy_30: 1-6

In [141]:
df_tempo = pd.read_json('taz.vz_tbl_tempo_l.json')

In [142]:
from pandas.io.json import json_normalize
strassen = json_normalize(df_tempo['features'])
strassen.head()

,type,geometry.type,geometry.coordinates,properties.objid,properties.messwert_von,properties.messwert_bis,properties.vmax,properties.umsetzung,properties.laenge,properties.vmax_soll_grund,properties.gemeldetvon,properties.lokalisationsname,properties.lokalisationnummer,properties.t30_typ
0,Feature,LineString,"[[8.5281326185, 47.4126736729], [8.5282732659,...",3603618,0.000000,130.641502,30,None,None,None,None,Zelghalde,2239,1
1,Feature,LineString,"[[8.5098825374, 47.3778438034], [8.5096958626,...",3603619,34.490748,144.897842,30,None,None,None,None,Hardaustrasse,779,1
2,Feature,LineString,"[[8.5443069125, 47.4224801012], [8.5443129737,...",3603620,0.000000,328.484562,0,None,None,None,None,namenlos164,7164,1
3,Feature,LineString,"[[8.5146172328, 47.3703809404], [8.5143827054,...",3603621,1147.008000,1165.030000,30,20190612000000,None,None,DAV,Birmensdorferstrasse,235,1
4,Feature,LineString,"[[8.5490012779, 47.4240972205], [8.5489130507,...",3603622,0.000000,373.818464,0,None,None,None,None,namenlos294,7294,1


In [143]:
strassen.iloc[1]['geometry.coordinates']

[[8.5098825374, 47.3778438034],
 [8.5096958626, 47.3777034879],
 [8.5096007507, 47.3776136453],
 [8.5089157237, 47.3770997631]]

In [144]:
# https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas
import numpy as np
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

4.842620155129374

In [145]:
print(strassen.iloc[1]['properties.lokalisationsname'])
df_coord = pd.DataFrame(strassen.iloc[1]['geometry.coordinates'])
# df_coord = df_coord.transpose()
df_coord.columns = ['long', 'lat']
df_coord

Hardaustrasse


,long,lat
0,8.509883,47.377844
1,8.509696,47.377703
2,8.509601,47.377614
3,8.508916,47.377100


In [146]:
df_coord['dist'] = \
    haversine_np(df_coord.long.shift(), df_coord.lat.shift(),
                 df_coord.loc[1:, 'long'], df_coord.loc[1:, 'lat'])
df_coord

,long,lat,dist
0,8.509883,47.377844,NaN
1,8.509696,47.377703,0.020987
2,8.509601,47.377614,0.012284
3,8.508916,47.377100,0.076930


In [147]:
# https://www.google.com/maps/dir/'47.413287,8.475857'/'47.412267,8.468761'
df_coord.dist.sum()

0.11020126947115666

In [174]:
df_tempo.features.iloc[1]['geometry']['coordinates']
# namenlos62
# https://www.google.com/maps/dir/'47.4132965029,8.5668369019'/'47.4156848433,8.5662862452'


[[8.5098825374, 47.3778438034],
 [8.5096958626, 47.3777034879],
 [8.5096007507, 47.3776136453],
 [8.5089157237, 47.3770997631]]

In [179]:
for strasse in df_tempo.features.iteritems():
    if strasse[1]['properties']['lokalisationsname'].startswith('namenlos'):
        continue
    # TODO: merge the Line and MultilLine branches
    if strasse[1]['geometry']['type'] == 'LineString':
        df_coord = pd.DataFrame(strasse[1]['geometry']['coordinates'])
        df_coord.columns = ['long', 'lat']
        df_coord['dist'] = \
        haversine_np(df_coord.long.shift(), df_coord.lat.shift(),
                     df_coord.loc[1:, 'long'], df_coord.loc[1:, 'lat'])
        print(strasse[1]['properties']['lokalisationsname'], df_coord.dist.sum())
    else: # MultiLineString
        summe = 0
        for strecke in strasse[1]['geometry']['coordinates']:  
            df_coord = pd.DataFrame(strecke)
            df_coord.columns = ['long', 'lat']
            df_coord['dist'] = \
            haversine_np(df_coord.long.shift(), df_coord.lat.shift(),
                         df_coord.loc[1:, 'long'], df_coord.loc[1:, 'lat'])
            summe += df_coord.dist.sum()
        print(strasse[1]['properties']['lokalisationsname'], summe)
    # print(strasse[1]['geometry']['coordinates'])
    

Zelghalde 0.13043469526801704
Hardaustrasse 0.11020126947115666
Birmensdorferstrasse 0.017959324952704487
Eisfeldstrasse 0.08549771964500383
Mainaustrasse 0.17633427081074232
Talbächliweg 0.13521738079109966
Bruderwies 0.036115517745224056
Im Struppen 0.1717469239414816
Bucheggstrasse 0.16540405384455276
Herzogenmühlestrasse 0.5642095899925281
Forchstrasse 0.5569511578845218
Im Schellenberg 0.04544823544538228
Winzerhalde 0.27243513633003924
Hofstrasse 0.09245185887989607
Hohensteinweg 0.33802266855550045
Leonhard-Ragaz-Weg 0.05469879489276762
Paul-Clairmont-Strasse 0.3487252800614404
Neunbrunnenstrasse 0.08387388049520293
Gubelhangstrasse 0.06260973376514249
Forrenweidstrasse 0.36098006686882156
Jakob-Fügli-Strasse 0.1541721728165931
Langgrütstrasse 0.06946257150862716
Bachtelweg 0.029670819061604892
Kleinalbis 0.18344570979773683
Mittelstrasse 0.052220311346325814
Gartenweg 0.07034376507661312
Ohmstrasse 0.2686417774254778
Hallerweg 0.13355415959271677
Grubenstrasse 0.665367983331283

Ceresstrasse 0.08751388269422852
Affolternstrasse 0.14690452938168147
Rennweg 0.208328924838314
Engweg 0.09082922331180582
Oetenbachgasse 0.08314045502187693
Maienweg 0.2415971958768849
Genossenschaftsstrasse 0.1279297728491175
Gerstenstrasse 0.12215253603144442
Tram Aubrugg 0.11595216868716111
Burstwiesenstrasse 0.2544828955911041
Schaffhauserstrasse 1.082265816544528
Allmannstrasse 0.4462007172175683
James-Joyce-Strasse 0.052689279313114123
Tiefenhöfe 0.09495398350864187
Oetenbachgasse 0.15192840614061842
Bocklerstrasse 0.05234389944980147
Zimmergasse 0.177209082544903
Bruneggweg 0.06828455598488925
Napfgasse 0.06253000161586952
Zeltweg 0.6851593652998037
Distelweg 0.25124609752007
Holzhofweg 0.07849118572817598
Weststrasse 0.1141760505264105
Hirschengraben 0.4275723114419083
Culmannstrasse 0.0025831412828966967
Maneggstrasse 0.6542143866624568
Staudenbühlstrasse 0.039327814400303356
Birchsteg 0.14460859084235494
Eisenbahnerstrasse 0.19261167228203951
Elias-Canetti-Strasse 0.10486716

Friedheimstrasse 0.08539975996179057
Klusdörfli 0.06242764489978182
Teufwiesenstrasse 0.07477577958076463
Hardbrücke 0.3708296073814998
Herbstweg 0.6815531976438614
Schaufelbergerstrasse 0.02873362014961962
Milchbuckstrasse 0.09141450942173951
Kleinjoggsteig 0.3131274030825092
Emil-Oberhänsli-Anlage 0.006813663581113527
Stephan-à-Porta-Weg 0.07173693211669419
Kanalstrasse 0.21479278766976817
Tram Bürkliplatz 0.1506720279231895
Streulistrasse 0.2186526970513185
Wasserwerkstrasse 0.6478428446279454
Blaumeisliweg 0.0621307546692504
Schürbungert 0.22333925199096888
Talwiesenstrasse 0.07663357987048715
Laubfroschweg 0.1406796241668393
Leutschenbachstrasse 0.30460791446654273
Grundstrasse 0.03380272916378231
Brauerstrasse 0.2653001503637355
Bahnhaldenstrasse 0.042091666972156994
Spindelstrasse 0.1035303616158873
Langstrasse 0.21804110988783773
FIFA-Strasse 0.14417475605532548
Birchstrasse 0.08235466404003783
Loorengutweg 0.5150220239836792
Bahnhaldenstrasse 0.09991872468304391
Klausstrasse 0

Kühriedweg 0.22856653123930568
Seeblickstrasse 0.3089501654750675
Zurlindenstrasse 0.1307056685632279
Adolf-Lüchinger-Strasse 0.14884930934903734
Joachim-Hefti-Weg 0.15310974704029046
Allenmoosstrasse 0.15710984074229994
Ruedi-Walter-Strasse 0.24652406033434468
Schubertstrasse 0.2206142332014143
Simmlersteig 0.16926197482754063
Kinkelstrasse 0.07082079779353931
Goldbrunnenplatz 0.08703447736239994
Ankerstrasse 0.22004437081512468
Zeppelinstrasse 0.008118608381685854
A3W Autobahnschlaufe West 0.3508729748883414
Soodstrasse 0.8558543062811842
Emil-Oprecht-Platz 0.08753830501904243
Im Trumeter 0.15063441351402188
Utobrücke 0.06477082923388107
Tannenrauchstrasse 0.7270089506021029
Wallisellenstrasse 0.11629151407749864
Gagliardiweg 0.001035062117433399
Käferholzstrasse 0.5448456491441168
Affolternstrasse 0.08202868884979314
Haldenstrasse 0.03212821427447959
Rigiplatz 0.14166873181534476
Schärenmoosstrasse 0.33244143511907626
Segetenweg 0.7036315066998918
Meierwiesenstrasse 0.08139775233778

Ernst-Laur-Hof 0.017861401968918278
Schwamendingerplatz 0.015836552686917027
Dunkelhölzlistrasse 0.4296466735548683
Werdhölzlistrasse 0.17980809413216764
Hurdäckerstrasse 0.09388552857728508
Schulstrasse 0.08882946245214657
Bühlholzweg 0.5780083544707575
Manesseplatz 0.033563457064725966
Pappelstrasse 0.22597686008888945
Anton-Higi-Strasse 0.12488595807961511
Unterwerkstrasse 0.15932465704278154
Gmeimeriweg 0.1315988330570349
Sonnenrain 0.10937277276951539
Wegackerstrasse 0.10542395348260399
Meier-Bosshard-Strasse 0.12964819422539453
Kunzweg 0.09004889379775144
Hinterbergstrasse 0.24454874474240654
General-Guisan-Quai 0.11462530339393956
Hirzenbachstrasse 0.21945674844069404
Reckenholzstrasse 0.1026007536785658
Seebacherstrasse 0.0983434049341384
Basteiplatz 0.1645607287068565
Schönauweg 0.12873263901723378
Riesbachstrasse 0.11167072940057253
Heimplatz 0.056114701652569335
Schulhausstrasse 0.1043407908000235
Breitensteinstrasse 0.7672273651925112
Cordelia-Guggenheim-Weg 0.1196131632151

...

A3W 1.1823036129581117
Rainstrasse 0.024693305234784378
Apfelbaumstrasse 0.07471059939893578
Hitzigweg 0.06367699340219955
Oerlikerpark 0.061267483402992304
Regulastrasse 0.21642374475580267
Bernerstrasse 0.6749254440186341
Schönauweg 0.09717431458476533
Hirschwiesenstrasse 0.10163383204775073
Pfirsichstrasse 0.1356558890792875
Veilchenstrasse 0.1938043582678123
Röschibachplatz 0.026615597217755246
Frymannstrasse 0.4998625425098628
Leutschenbachstrasse 0.4587678985030121
Stierliweg 0.11970568941855446
Regensdorferstrasse 0.27452985451437206
Seebacherstrasse 0.04283832424713178
Sihlbrücke 0.062205650944690556
Wiesenstrasse 0.19855987344557507
Hugo-Koblet-Weg 0.16581792892271802
Dachslernstrasse 0.13329695422336035
Birchstrasse 0.035857938386034044
Bahnhofquai 0.22051307456744199
Gablerstrasse 0.02483364885729714
Gerbergasse 0.013497392780687307
Luisenstrasse 0.07589453189290692
Tièchestrasse 0.06051075839270722
Kappelistrasse 0.39439333275282007
Auguste-Piccard-Hof 0.14061466398800004
N

Nüschelerstrasse 0.10316112789278664
Mühlebachstrasse 0.3071955531040727
Luchswiesenstrasse 0.058360886547819214
Pfingstweid 0.0044913178135412395
Buchfinkenstrasse 0.12453286557666617
Döltschiweg 0.0956960980691996
In der Ey 0.0876566193450103
Blumenfeldstrasse 0.3265009855833795
Im Holzerhurd 0.050661480318221756
Mühlebachstrasse 0.6522958256121759
Edelweissstrasse 0.11051052264652005
Grüngasse 0.2967500532890482
Usteristrasse 0.053182973409557896
Regulastrasse 0.07164438229085389
Steinkluppenweg 0.1459885739266923
Pelikanplatz 0.03748482575359175
Werdstrasse 0.4394490826009422
Flurstrasse 0.15023788052090153
Kinkelstrasse 0.15163659811300806
Röntgenplatz 0.0317230541981149
Klosterweg 0.029009815110858
Nordheimstrasse 0.26107158798347474
Ampèresteg 0.08357075381471184
Katzenbachweg 0.19438559187343915
Gamperstrasse 0.061652489833306866
Schützenrain 0.23977864663773324
Fröbelstrasse 0.08856905064780131
Widdergasse 0.04483542059531957
Am Suteracher 0.12823414685013987
Buchmattweg 0.104

Oerlikonerstrasse 0.9404759141831243
Schäppistrasse 0.19643657066398504
Obere Bläsistrasse 0.07793417829823746
Regensdorferstrasse 0.03658857448292037
Küngenmatt 0.19559572770010525
Wässerlingweg 0.41053961812795425
Venusstrasse 0.09829330236170898
Wengistrasse 0.12333322875778155
Fritz-Fleiner-Weg 0.04175453824928958
Kloster-Fahr-Weg 4.773179848694148
Bergliweg 0.11287176292985177
Kempfhofsteig 0.16424568813503548
Leonhardsteig 0.0701374259893204
Gubelhangstrasse 0.0585470232202104
Frohburgstrasse 0.7760994911762321
Kolbenacker 0.0305456318180007
Kalchbühlstrasse 0.4223208227029339
Speicherweg 0.9403598199480123
Brunaustrasse 0.5501805716928929
Wolframplatz 0.059641726192880946
A1L 0.08084328770881835
Waidhofweg 0.8043807861042752
Hintermeisterhof 0.10037792293184396
Probsteistrasse 0.5691874869032635
Jonas-Furrer-Strasse 0.2206127206135359
Frymannstrasse 0.40946199570414044
Farbhofstrasse 0.133481198435663
A3 0.23709030370419865
Schwingerstrasse 0.0640796332633804
Ulrichstrasse 0.087

Schöneichtunnel 0.6380327026819385
Zehntenhausplatz 0.08099306132712239
Englischviertelstrasse 0.2220176397570482
Arianestrasse 0.06438008601760806
Technoparkstrasse 0.21660399121459684
Turbinenplatz 0.10022690875450885
Unterholzweg 0.24979020866081553
Wengistrasse 0.020595920389258365
Fürstweg 0.1180028152764272
Im Schellenberg 0.06506167909288678
Herbartstrasse 0.0817415880639816
Gratweg 0.8499148509762645
Münchhaldenstrasse 0.1276741191310735
Giessereistrasse 0.2264214786115844
Im Brächli 0.18752905551894675
Bühlstrasse 0.2260880556809093
Bucheggstrasse 0.1071869989425816
Wohllebgasse 0.10993620259417536
Bellerivestrasse 0.27218058358875136
Aubrigstrasse 0.19194834985994585
Eichraintunnel 0.22113704633925804
Nansenstrasse 0.05884394453928459
Altwiesenstrasse 1.3713002246745458
Herostrasse 0.22187694551811815
Möhrlistrasse 0.6860666414490322
Arbentalstrasse 0.20693805571953383
Geerenweg 0.22039337178767338
Wallisellenstrasse 0.10000708262987434
Buchenweg 0.20660535554516377
Max-Hunzi

Bombachhalde 0.23330291409225362
Ulmbergtunnel 0.008089438311838839
Andreasstrasse 0.32499428840301253
Waidbergweg 0.6076139939295079
Zwyssigstrasse 0.2000214928404471
Sackzelg 0.3195008720713674
Robert-Gnehm-Weg 0.1510644112531328
Meinrad-Lienert-Strasse 0.1838510117572238
Frohalpstrasse 0.21578638117738191
Busrampe ETH Hönggerberg 0.10094782830457151
Widumweg 0.07550918834790267
Bernerstrasse Süd 0.16251814120079344
Gubelplatz 0.009028755041934584
Widmerstrasse 0.3314014092506417
Seebahnstrasse 1.1885141087231141
Saatlenstrasse 0.2120442886170233
Rüdenplatz 0.05894681616529869
Turnerstrasse 0.5803188327193741
Kirchenfeld 0.480706952963506
Bodenacker 0.20626899763212336
Stotzstrasse 0.041131313485654124
St. Annagasse 0.11585972127722768
Beckenhofstrasse 0.6228481878128032
Alfred-Escher-Strasse 0.1283595164906575
Kurvenstrasse 0.2687282950052667
Stelzenstrasse 0.20055615538622135
Walcheplatz 0.014242371809803818
Leimbachweg 0.4433252486286707
Unterfeldstrasse 0.3380866454535148
Austras